<img src="img/pandora2d_logo.png" width="500" height="500">

# Pandora2D : a coregistration framework

# Introduction and basic usage

#### Imports and external functions

In [ ]:
import pandora
import io
from pathlib import Path
from IPython.display import Image, display

In [ ]:
def plot_state_machine(pandora2d_machine):
    """
    Show the schemes of step of Pandora2D Machine
    """
    stream = io.BytesIO()
    try:
        pandora2d_machine.get_graph().draw(stream, prog='dot', format='png')
        display(Image(stream.getvalue()))
    except:
        print("It is not possible to show the graphic of the state machine. To solve it, please install graphviz on your system (apt-get install graphviz if operating in Linux) and install python package with pip insall graphviz")

In [ ]:
from snippets.utils import *

# What is Pandora2D ?
* Pandora2d is a Toolbox to estimate disparity in two dimensions
* It works with Pandora
* It is easy to configure and modular

## Inputs

* Image pair
* Value associated to no_data images
* Disparity ranges to explore 
* Configuration file

## Outputs

* Disparity maps for row's axis
* Disparity maps for column's axis

## Pandora2D's pipeline

Pandora2D provides the following steps:
* matching cost computation (**mandatory**)
* disparity computation (**mandatory**)
* subpixel disparity refinement

<img src="img/Pandora2D_pipeline.png" width="700">

### Available implementations for each step

| Step                      | Algorithms implemented |
|:--------------------------|:-----------------------|
| Matching cost computation | SAD / SSD / ZNNC       |
| Disparity computation     | Winner-Takes-All        |
| Subpixel disparity refinement | Interpolation      |

# Pandora2D execution options with state machine

#### Imports of pandora2d

In [ ]:
# Load pandora imports
import pandora2d
from pandora2d.state_machine import Pandora2DMachine
from pandora2d import check_json


from pandora.img_tools import read_img

#### Load and visualize input data 

Provide image path

In [ ]:
# Paths to left and right images
img_left_path = "data/left.tif"
img_right_path = "data/right.tif"

In [ ]:
# image configuration
image_cfg = {'image': {'no_data_left': np.nan, 'no_data_right': np.nan}}

Provide output directory to write results

In [ ]:
output_dir = os.path.join(os.getcwd(),"output")
# If necessary, create output dir
Path(output_dir).mkdir(exist_ok=True,parents=True)

Convert input data to dataset

In [ ]:
img_left = read_img(img_left_path, no_data=image_cfg['image']['no_data_left'])
img_right = read_img(img_right_path, no_data=image_cfg['image']['no_data_right'])

Visualize input data

In [ ]:
fig = plt.figure(figsize=(10,10))
ax0 = fig.add_subplot(1,2,1)
ax0.imshow(img_left["im"].data)
plt.title("Left image")
ax1 = fig.add_subplot(1,2,2)
ax1.imshow(img_right["im"].data)
plt.title("Right image")

# Option 1 : trigger all the steps of the machine at ones

#### Instantiate the machine

In [ ]:
pandora2d_machine = Pandora2DMachine()

#### Define pipeline configuration

In [ ]:
user_pipeline_cfg = {
    'pipeline':{
        "matching_cost" : {
            "matching_cost_method": "zncc",
            "window_size": 5,
        },
        "disparity": {
            "disparity_method": "wta",
            "invalid_disparity": -2
        },
        "refinement" : {
            "refinement_method" : "interpolation"
        }
    }
}

Disparity interval used for row

In [ ]:
disp_min_row = -2
disp_max_row = 2

Disparity interval used for colummns

In [ ]:
disp_min_col = -2
disp_max_col = 2

#### Check the configuration and sequence of steps

In [ ]:
checked_cfg = check_json.check_pipeline_section(user_pipeline_cfg, pandora2d_machine)

In [ ]:
pipeline_cfg = checked_cfg['pipeline']
print(pipeline_cfg)

#### Prepare the machine 

In [ ]:
pandora2d_machine.run_prepare(img_left, img_right, disp_min_col, disp_max_col, disp_min_row, disp_max_row)

#### Trigger all the steps of the machine at ones

In [ ]:
dataset = pandora2d.run(pandora2d_machine, img_left, img_right, disp_min_col, disp_max_col, disp_min_row, disp_max_row, pipeline_cfg)

Visualize output disparity map

In [ ]:
plot_image(dataset["row_map"].data, "Row refined disparity map", output_dir, cmap=pandora_cmap())

In [ ]:
plot_image(dataset["col_map"].data, "Columns refined disparity map", output_dir, cmap=pandora_cmap())

# Option 2 : trigger the machine step by step

The implementation of Pandora2D with a state machine makes it possible to set up a more flexible pipeline, which makes it possible to choose via a configuration file the steps wishes to follow in Pandora2D.

Moreover, the state machine allows to run each step of the pipeline independently, giving the possibility to save and visualize the results after each step. 

The state machine has three states : 
* Begin
* Cost volumes
* Disparity maps

Being the connections between them the different steps of the pipeline.

<img src="img/Pandora2D_pipeline.png" width="700">

#### Instantiate the machine

In [ ]:
pandora2d_machine = Pandora2DMachine()

#### Define pipeline configuration

In [ ]:
user_pipeline_cfg = {
    'pipeline':{
        "matching_cost" : {
            "matching_cost_method": "zncc",
            "window_size": 5,
        },
        "disparity": {
            "disparity_method": "wta",
            "invalid_disparity": -5
        },
        "refinement":{
            "refinement_method" : "interpolation"
        }
    }
}

Disparity interval used for row

In [ ]:
disp_min_row = -2
disp_max_row = 2

Disparity interval used for colummns

In [ ]:
disp_min_col = -2
disp_max_col = 2

#### Check the configuration and sequence of steps

In [ ]:
checked_cfg = check_json.check_pipeline_section(user_pipeline_cfg, pandora2d_machine)

In [ ]:
pipeline_cfg = checked_cfg['pipeline']
print(pipeline_cfg)

#### Prepare the machine 

In [ ]:
pandora2d_machine.run_prepare(img_left, img_right, disp_min_col, disp_max_col, disp_min_row, disp_max_row)

#### Trigger the machine step by step

In [ ]:
plot_state_machine(pandora2d_machine)

Run matching cost 

In [ ]:
pandora2d_machine.run('matching_cost', pipeline_cfg)

In [ ]:
plot_state_machine(pandora2d_machine)

Run disparity 

In [ ]:
pandora2d_machine.run('disparity', pipeline_cfg)

In [ ]:
plot_state_machine(pandora2d_machine)

In [ ]:
plot_image(pandora2d_machine.dataset_disp_maps["col_map"].data, "Columns disparity map", output_dir, cmap=pandora_cmap())

In [ ]:
plot_image(pandora2d_machine.dataset_disp_maps["row_map"].data, "Row disparity map", output_dir, cmap=pandora_cmap())

Run refinement 

In [ ]:
pandora2d_machine.run('refinement', pipeline_cfg)

In [ ]:
plot_state_machine(pandora2d_machine)

Visualize output disparity map

In [ ]:
plot_image(pandora2d_machine.dataset_disp_maps["col_map"].data, "Columns refined disparity map", output_dir, cmap=pandora_cmap())

In [ ]:
plot_image(pandora2d_machine.dataset_disp_maps["row_map"].data, "Row refined disparity map", output_dir, cmap=pandora_cmap())